In [1]:
!nvidia-smi

Thu Nov 10 05:33:04 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   56C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [4]:
#copy the files saved in drive to Colab working directory. Links to download the paths are in the comments below
# SE_checkpoint.pth.tar = "https://huggingface.co/spaces/BilalSardar/Voice-Cloning/resolve/main/SE_checkpoint.pth.tar"
# best_model_latest.pth.tar = "https://huggingface.co/spaces/BilalSardar/Voice-Cloning/resolve/main/best_model_latest.pth.tar"
!cp '/content/gdrive/MyDrive/Voice-Cloning-Coqui/SE_checkpoint.pth.tar' 'SE_checkpoint.pth.tar'
!cp '/content/gdrive/MyDrive/Voice-Cloning-Coqui/best_model_latest.pth.tar' 'best_model_latest.pth.tar'

In [3]:
!git clone https://github.com/athenasaurav/Coqui-App.git

Cloning into 'Coqui-App'...
remote: Enumerating objects: 10, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 10 (delta 0), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (10/10), done.


In [5]:
%cd Coqui-App

/content/Coqui-App


In [6]:
!git clone https://github.com/Edresson/Coqui-TTS -b multilingual-torchaudio-SE TTS
!pip install -q -e TTS
!pip install -q torchaudio==0.9.0
!pip install gradio
!pip install -q pydub ffmpeg-normalize

Cloning into 'TTS'...
remote: Enumerating objects: 25680, done.
remote: Total 25680 (delta 0), reused 0 (delta 0), pack-reused 25680
Receiving objects: 100% (25680/25680), 133.78 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (18753/18753), done.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 125 kB 33.6 MB/s 
     |████████████████████████████████| 80 kB 9.5 MB/s 
     |████████████████████████████████| 1.4 MB 45.0 MB/s 
     |████████████████████████████████| 287 kB 68.8 MB/s 
     |████████████████████████████████| 183 kB 71.9 MB/s 
     |████████████████████████████████| 3.4 MB 55.1 MB/s 
     |████████████████████████████████| 71 kB 9.2 MB/s 
     |████████████████████████████████| 47.4 MB 1.2 MB/s 
     |████████████████████████████████| 11.1 MB 21.6 MB/s 
     |████████████████████████████████| 487 kB 55.0 MB/s 
     |████████████████████████████████| 25.3 MB 1

In [7]:
from turtle import title
import gradio as gr

# import git
import os
# os.system('git clone https://github.com/Edresson/Coqui-TTS -b multilingual-torchaudio-SE TTS')
# os.system('pip install -q -e TTS/')
# os.system('pip install -q torchaudio==0.9.0')

import sys
TTS_PATH = "TTS/"

# add libraries into environment
sys.path.append(TTS_PATH) # set this if TTS is not installed globally

import os
import string
import time
import argparse
import json

import numpy as np
import IPython
from IPython.display import Audio


import torch

In [8]:
from TTS.tts.utils.synthesis import synthesis
from TTS.tts.utils.text.symbols import make_symbols, phonemes, symbols
try:
  from TTS.utils.audio import AudioProcessor
except:
  from TTS.utils.audio import AudioProcessor


from TTS.tts.models import setup_model
from TTS.config import load_config
from TTS.tts.models.vits import *  

OUT_PATH = 'out/'

# create output path
os.makedirs(OUT_PATH, exist_ok=True)

In [9]:
MODEL_PATH = '/content/best_model_latest.pth.tar'
CONFIG_PATH = '/content/Coqui-App/config.json'
TTS_LANGUAGES = "/content/Coqui-App/language_ids.json"
TTS_SPEAKERS = "/content/Coqui-App/speakers.json"
USE_CUDA = torch.cuda.is_available()  

In [10]:
# load the config
C = load_config(CONFIG_PATH)


# load the audio processor
ap = AudioProcessor(**C.audio)

speaker_embedding = None

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:80
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:False
 | > do_amp_to_db_mel:True
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024


In [11]:
C.model_args['d_vector_file'] = TTS_SPEAKERS
C.model_args['use_speaker_encoder_as_loss'] = False

model = setup_model(C)
model.language_manager.set_language_ids_from_file(TTS_LANGUAGES)

 > Using model: vits
 > Speaker manager is loaded with 6 speakers: female-en-5, female-en-5
, female-pt-4
, male-en-2, male-en-2
, male-pt-3



In [12]:
cp = torch.load(MODEL_PATH, map_location=torch.device('cpu'))

In [13]:
model_weights = cp['model'].copy()
for key in list(model_weights.keys()):
  if "speaker_encoder" in key:
    del model_weights[key]

model.load_state_dict(model_weights)


model.eval()

Vits(
  (emb_l): Embedding(3, 4)
  (text_encoder): TextEncoder(
    (emb): Embedding(165, 192)
    (encoder): RelativePositionTransformer(
      (dropout): Dropout(p=0.1, inplace=False)
      (attn_layers): ModuleList(
        (0): RelativePositionMultiHeadAttention(
          (conv_q): Conv1d(196, 196, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(196, 196, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(196, 196, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(196, 196, kernel_size=(1,), stride=(1,))
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (1): RelativePositionMultiHeadAttention(
          (conv_q): Conv1d(196, 196, kernel_size=(1,), stride=(1,))
          (conv_k): Conv1d(196, 196, kernel_size=(1,), stride=(1,))
          (conv_v): Conv1d(196, 196, kernel_size=(1,), stride=(1,))
          (conv_o): Conv1d(196, 196, kernel_size=(1,), stride=(1,))
          (dropout): Dropout(p=0.1, inplace=False)
        )
        (2): Rela

In [14]:
if USE_CUDA:
    model = model.cuda()

# synthesize voice
use_griffin_lim = False

In [15]:
CONFIG_SE_PATH = "/content/Coqui-App/config_se.json"
CHECKPOINT_SE_PATH = "/content/SE_checkpoint.pth.tar"

In [16]:
from TTS.tts.utils.speakers import SpeakerManager
from pydub import AudioSegment
import librosa


In [20]:
SE_speaker_manager = SpeakerManager(encoder_model_path=CHECKPOINT_SE_PATH, encoder_config_path=CONFIG_SE_PATH, use_cuda=USE_CUDA)

 > Setting up Audio Processor...
 | > sample_rate:16000
 | > resample:False
 | > num_mels:64
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:512
 | > power:1.5
 | > preemphasis:0.97
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:False
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:False
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > stats_path:None
 | > base:10
 | > hop_length:160
 | > win_length:400


In [21]:
def compute_spec(ref_file):
  y, sr = librosa.load(ref_file, sr=ap.sample_rate)
  spec = ap.spectrogram(y)
  spec = torch.FloatTensor(spec).unsqueeze(0)
  return spec

In [22]:
def greet(Text,Voicetoclone):
    text= "%s" % (Text)
    reference_files= "%s" % (Voicetoclone)
    print("path url")
    print(Voicetoclone)
    sample= str(Voicetoclone)
    size= len(reference_files)*sys.getsizeof(reference_files)
    size2= size / 1000000
    if (size2 > 0.012) or len(text)>2000:
      message="File is greater than 30mb or Text inserted is longer than 2000 characters. Please re-try with smaller sizes."
      print(message)
      raise SystemExit("File is greater than 30mb. Please re-try or Text inserted is longer than 2000 characters. Please re-try with smaller sizes.")
    else:
      os.system('ffmpeg-normalize $sample -nt rms -t=-27 -o $sample -ar 16000 -f')
      reference_emb = SE_speaker_manager.compute_d_vector_from_clip(reference_files)
      model.length_scale = 1.5  # scaler for the duration predictor. The larger it is, the slower the speech.
      model.inference_noise_scale = 0.3 # defines the noise variance applied to the random z vector at inference.
      model.inference_noise_scale_dp = 0.3 # defines the noise variance applied to the duration predictor z vector at inference.
      text = text
      model.language_manager.language_id_mapping
      language_id = 0
    
      print(" > text: {}".format(text))
      wav, alignment, _, _ = synthesis(
                        model,
                        text,
                        C,
                        "cuda" in str(next(model.parameters()).device),
                        ap,
                        speaker_id=None,
                        d_vector=reference_emb,
                        style_wav=None,
                        language_id=language_id,
                        enable_eos_bos_chars=C.enable_eos_bos_chars,
                        use_griffin_lim=True,
                        do_trim_silence=False,
                    ).values()
      print("Generated Audio")
      IPython.display.display(Audio(wav, rate=ap.sample_rate))
      file_name = text.replace(" ", "_")
      file_name = file_name.translate(str.maketrans('', '', string.punctuation.replace('_', ''))) + '.wav'
      out_path = os.path.join(OUT_PATH, file_name)
      print(" > Saving output to {}".format(out_path))
      ap.save_wav(wav, out_path)
      return out_path

In [ ]:
# demo = gr.Interface(
#     fn=greet, 
#     inputs=[gr.inputs.Textbox(label='What would you like the voice to say? (max. 2000 characters per request)'),gr.Audio(type="filepath",         source="upload",label='Please upload a voice to clone (max. 30mb)')],
#     outputs="audio",
#     title="Bilal's Voice Cloning Tool"
#     )
# demo.launch(share=True)

/usr/local/lib/python3.7/dist-packages/gradio/inputs.py:27: UserWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  "Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components",
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `optional` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: `numeric` parameter is deprecated, and it has no effect
  warnings.warn(value)
/usr/local/lib/python3.7/dist-packages/gradio/deprecation.py:40: UserWarning: The 'type' parameter has been deprecated. Use the Number component instead.
  warnings.warn(value)


Colab notebook detected. To show errors in colab notebook, set `debug=True` in `launch()`
Running on public URL: https://b054a61187f5a7f4.gradio.app

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


(<gradio.routes.App at 0x7fe38a6ff950>,
 'http://127.0.0.1:7861/',
 'https://b054a61187f5a7f4.gradio.app')

In [25]:
greet("Today i want to tell you seven poems i like", "/content/intrested.wav")

path url
/content/intrested.wav
 > text: Today i want to tell you seven poems i like
Generated Audio


'out/Today_i_want_to_tell_you_seven_poems_i_like.wav'

 > Saving output to out/Today_i_want_to_tell_you_seven_poems_i_like.wav
